In [1]:
import tensorflow as tf
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import timeit

In [2]:
filenames = []
for file in os.listdir('audio'):
    filenames.append(file)
# filenames[0:2]

In [3]:
random.shuffle(filenames)

In [4]:
max, min = 76117, 53310
start, end = 21000, 53310
wave_length = end-start
num_train = int(len(filenames)*0.7)
num_test = len(filenames) - num_train

In [5]:
data_x_train = []
data_x_test = []

start_time = timeit.default_timer()
for filename in filenames[:num_train]:
    data, sampling_rate = librosa.load("audio/" + filename, res_type='kaiser_fast')
    data_x_train.append(data[start:end])
    
for filename in filenames[num_train:]:
    data, sampling_rate = librosa.load("audio/" + filename, res_type='kaiser_fast')
    data_x_test.append(data[start:end])
    
stop_time = timeit.default_timer()
print('runtime: ', stop_time - start_time)  

runtime:  15.625957359164424


In [7]:
# max = 0
# min = 57117
# for i in range(len(data_x)):
#     if len(data_x[i]) > max:
#         max = len(data_x[i])
#     if len(data_x[i]) < min:
#         min = len(data_x[i])
# print("max =", max, ",min =", min, ",length =",wave_length, ",len(data_x) =",len(data_x))

In [8]:
data_y_train = []
data_y_test = []

for filename in filenames[:num_train]:
    if filename[7] == '3':
        data_y_train.append(1)
    elif filename[7] == '4':
        data_y_train.append(0)
        
for filename in filenames[num_train:]:
    if filename[7] == '3':
        data_y_test.append(1)
    elif filename[7] == '4':
        data_y_test.append(0)   

In [9]:
# takes every value in data_y to an array
# ex: [0, 1, 1, 1, 0] => [[0], [1], [1], [1], [0]]
data_y_train_correct = []
data_y_test_correct = []

for val in data_y_train:
    val_arr = []
    val_arr.append(val)
    data_y_train_correct.append(val_arr)
    
for val in data_y_test:
    val_arr = []
    val_arr.append(val)
    data_y_test_correct.append(val_arr)

In [10]:
data_xy_train = []
for i in range(len(data_x_train)):
    data_xy_train.append( (data_x_train[i], data_y_train_correct[i]) )

In [11]:
def getXvalues(data_xy):
    x_values = []
    for data in data_xy:
        x_values.append(data[0])
    return x_values

def getYvalues(data_xy):
    y_values = []
    for data in data_xy:
        y_values.append(data[1])
    return y_values

In [12]:
def logistic_fun(z):
    return 1/(1.0 + np.exp(-z))

In [13]:
data_xy_train

[(array([ 1.1531988e-06,  6.2585792e-07, -1.7419092e-05, ...,
          3.1669522e-03,  2.7591526e-03,  2.3047454e-03], dtype=float32), [0]),
 (array([-0.00039403, -0.00037298, -0.00035394, ..., -0.00213083,
         -0.0132186 , -0.02187038], dtype=float32), [1]),
 (array([-1.3878651e-05, -2.4654668e-05, -4.6667370e-05, ...,
         -1.9563622e-05, -1.3774800e-05, -5.4347660e-05], dtype=float32), [0]),
 (array([ 7.1292707e-05,  6.2392588e-05, -1.2964624e-04, ...,
          2.9823028e-03,  3.0859837e-03,  3.2056735e-03], dtype=float32), [1]),
 (array([ 0.00016648,  0.00014982,  0.00014847, ..., -0.00018326,
         -0.00015046, -0.00011587], dtype=float32), [0]),
 (array([ 6.4281732e-05,  9.0964268e-06,  5.1705738e-05, ...,
         -2.0583846e-01, -2.2120658e-01, -2.2620307e-01], dtype=float32), [0]),
 (array([-9.6337862e-06, -6.6305515e-06, -7.4794743e-06, ...,
         -2.3801815e-02, -2.4177786e-02, -2.4059162e-02], dtype=float32), [0]),
 (array([0.00024708, 0.00024671, 0.0002075

In [14]:
features = wave_length
eps = 1e-12

x = tf.placeholder(tf.float32, [None, features])

y_ = tf.placeholder(tf.float32, [None, 1])

W = tf.Variable(tf.zeros([features,1]))

b = tf.Variable(tf.zeros([1]))

y = 1 / (1.0 + tf.exp(-(tf.matmul(x,W) + b)))

loss1 = -(y_ * tf.log(y + eps) + (1 - y_) * tf.log(1 - y + eps))

loss = tf.reduce_mean(loss1)

update = tf.train.GradientDescentOptimizer(0.00001).minimize(loss)

sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(0,100):
    sess.run(update, feed_dict = {x:getXvalues(data_xy_train), y_:getYvalues(data_xy_train)}) #BGD
    random.shuffle(data_xy_train)
    

In [15]:
# # guessing random number from 0 to 1, ~50% success
# # our model should be at least as good as this!!!!
# right = 0
# last_x = len(data_x)
# for i in range(num_train, last_x):
#     pred = random.randint(0,1)
#     if data_y[i] == 0 and pred < 0.5:
#         right += 1
#     if data_y[i] == 1 and pred > 0.5:
#         right += 1
#     print("pred:",pred,"true:",data_y[i],"right:",right)

In [16]:
# data_x_train =          [arr1, arr2, arr3]       # data_x_test = [arr4, arr5]
# shuffle(data_x_train) = [arr2, arr3, arr1]
# data_y_train = [0, 1, 1] # data_y_test = [0, 1]

right = 0
for i in range(len(data_x_test)):
    pred = logistic_fun(np.matmul(data_x_test[i],sess.run(W)) + sess.run(b))
    if data_y_test[i] == 0 and pred < 0.5:
        right += 1
    if data_y_test[i] == 1 and pred > 0.5:
        right += 1
    print("pred:",pred,"true:",data_y_test[i],"right:",right)

pred: [0.50000155] true: 0 right: 0
pred: [0.5000017] true: 0 right: 0
pred: [0.5000029] true: 1 right: 1
pred: [0.4999901] true: 1 right: 1
pred: [0.50000113] true: 1 right: 2
pred: [0.5000005] true: 0 right: 2
pred: [0.5000021] true: 0 right: 2
pred: [0.49999762] true: 1 right: 2
pred: [0.5000008] true: 0 right: 2
pred: [0.5000006] true: 0 right: 2
pred: [0.49999565] true: 1 right: 2
pred: [0.4999944] true: 0 right: 3
pred: [0.500002] true: 0 right: 3
pred: [0.49998862] true: 1 right: 3
pred: [0.5000013] true: 1 right: 4
pred: [0.49997342] true: 1 right: 4
pred: [0.50000066] true: 0 right: 4
pred: [0.5000033] true: 0 right: 4
pred: [0.5000008] true: 1 right: 5
pred: [0.4999727] true: 1 right: 5
pred: [0.5000024] true: 0 right: 5
pred: [0.5000101] true: 0 right: 5
pred: [0.50000215] true: 0 right: 5
pred: [0.5000105] true: 1 right: 6
pred: [0.50000125] true: 0 right: 6
pred: [0.49999923] true: 1 right: 6
pred: [0.5000032] true: 1 right: 7
pred: [0.5000013] true: 1 right: 8
pred: [0.50

In [17]:
#with happy/sad files from first 4 actors we got 6 out of 20 correct, 30%
#with happy/sad files from first 8 actors we got 15 out of 39 correct, 38.46153846153847%
print("right:",right,"total:",num_test,"percent correct:",right/num_test*100)

right: 12 total: 39 percent correct: 30.76923076923077
